# AWS Bedrock Tutorial

Amazon Bedrock is a fully managed service that provides access to foundation models from various AI providers through a single API.

In this notebook we'll:
1. Use the **Converse API** - the simplest way to interact with Bedrock models
2. Compare two models on the same prompt:
   - **Amazon Nova Micro** - Amazon's fastest/cheapest text model
   - **Mistral 7B** - Popular open-source model

## Setup

Create a Bedrock client. Credentials are picked up automatically from `~/.aws/credentials`.

In [1]:
import boto3

bedrock = boto3.client("bedrock-runtime")

## Helper Function

A simple wrapper around the Converse API that also calculates cost.

In [3]:
# Pricing per 1K tokens (check aws.amazon.com/bedrock/pricing for current rates)
# Note: In EU regions, use inference profile IDs (eu.amazon.nova-*) instead of model IDs
PRICING = {
    "eu.amazon.nova-micro-v1:0": {"input": 0.000035, "output": 0.00014},
    "mistral.mistral-7b-instruct-v0:2": {"input": 0.00015, "output": 0.0002},
}


def ask_bedrock(model_id, prompt, max_tokens=512, temperature=0.7):
    """Send a prompt to Bedrock and return the response."""
    response = bedrock.converse(
        modelId=model_id,
        messages=[{"role": "user", "content": [{"text": prompt}]}],
        inferenceConfig={"maxTokens": max_tokens, "temperature": temperature},
    )

    text = response["output"]["message"]["content"][0]["text"]
    usage = response["usage"]

    # Calculate cost
    price = PRICING[model_id]
    cost = (usage["inputTokens"] * price["input"] + usage["outputTokens"] * price["output"]) / 1000

    print(f"Tokens: {usage['inputTokens']} in, {usage['outputTokens']} out")
    print(f"Cost: ${cost:.6f}")

    return text

## Test with Amazon Nova Micro

In [4]:
prompt = """Extract the following from this text and return as JSON:
- names (list) - in alphabetical order
- locations (list) - in alphabetical order
- dates (list) - use the same format as Python's datetime.isoformat()

Text: "John met Sarah in Paris on March 15th. They later traveled to London where they met Bob on April 2nd."
"""

In [5]:
print("=== Amazon Nova Micro ===")
print(ask_bedrock("eu.amazon.nova-micro-v1:0", prompt))

=== Amazon Nova Micro ===
Tokens: 80 in, 136 out
Cost: $0.000022
```json
{
  "names": [
    "Bob",
    "John",
    "Sarah"
  ],
  "locations": [
    "London",
    "Paris"
  ],
  "dates": [
    "2023-04-02T00:00:00",
    "2023-03-15T00:00:00"
  ]
}
```

Note: The dates are assumed to be in the year 2023, as the text does not specify the year. The time is set to 00:00:00 for simplicity.


## Test with Mistral 7B

In [6]:
print("=== Mistral 7B ===")
print(ask_bedrock("mistral.mistral-7b-instruct-v0:2", prompt))

=== Mistral 7B ===
Tokens: 97 in, 81 out
Cost: $0.000031
 {
  "names": ["John", "Sarah", "Bob"],
  "locations": ["Paris", "London"],
  "dates": ["2021-03-15T00:00:00", "2021-04-02T00:00:00"]
}


### Add format specification and date as context

In [7]:
import datetime

prompt = f"""Extract the following from this text and return as JSON:
- names (list in alphabetical order)
- locations (list in alphabetical order)
- dates (list) use the YYYY-MM-DD format

Respond with pretty-printed JSON content only.

Text: "John met Sarah in Paris on March 15th. They later traveled to London where they met Bob on April 2nd."

Context: The current year is {datetime.datetime.now().year}
"""

print(prompt)

Extract the following from this text and return as JSON:
- names (list in alphabetical order)
- locations (list in alphabetical order)
- dates (list) use the YYYY-MM-DD format

Respond with pretty-printed JSON content only.

Text: "John met Sarah in Paris on March 15th. They later traveled to London where they met Bob on April 2nd."

Context: The current year is 2025



In [8]:
print("=== Amazon Nova Micro ===")
print(ask_bedrock("eu.amazon.nova-micro-v1:0", prompt))

print("=== Mistral 7B ===")
print(ask_bedrock("mistral.mistral-7b-instruct-v0:2", prompt))

=== Amazon Nova Micro ===
Tokens: 92 in, 69 out
Cost: $0.000013
{
  "names": [
    "Bob",
    "John",
    "Sarah"
  ],
  "locations": [
    "London",
    "Paris"
  ],
  "dates": [
    "2025-04-02",
    "2025-03-15"
  ]
}
=== Mistral 7B ===
Tokens: 117 in, 89 out
Cost: $0.000035
 {
  "names": [
    "Bob",
    "John",
    "Sarah"
  ],
  "locations": [
    "Paris",
    "London"
  ],
  "dates": [
    "2025-03-15",
    "2025-04-02"
  ]
}
